# Loading required libraries

In [1]:
import re
import numpy as np
import csv
import pandas as pd

# Loading the LOINC file

In [2]:
# LOINC file/ at the time of writing the paper, the latest version was 2.78
csv_file = 'Loinc78.csv'

# Avoiding the mixed types error by setting the type of problematic ones
dtype_specification = {
    'VersionLastChanged': str,
    'EXMPL_ANSWERS': str,
    'SURVEY_QUEST_TEXT': str,
    'SURVEY_QUEST_SRC': str,
    'HL7_FIELD_SUBFIELD_ID': str,
    'STATUS_REASON': str,
    'STATUS_TEXT': str,
    'AskAtOrderEntry': str,
    'VersionFirstReleased': str,
    'ValidHL7AttachmentRequest': str 
}
# Loading the LOINC in a data frame
df = pd.read_csv(csv_file, dtype=dtype_specification)

# Filtering process

In [3]:
print('Total number of LOINCs: ', df.shape[0])

# The keywords we use for filtering out the LOINCs that may be associated with PII
fc = ['address', 'number', 'name', 'phone', 'fax', 'email', 'city', 'age', 'social security', 'longitude', 
      'latitude', 'identifier', 'date', 'zip', 'postal', 'license', 'certificate', 'url','county', 'town','telephone','nickname']

# Converting long_common_name to lowercase for case-insensitive search
df['long_common_name_lower'] = df['LONG_COMMON_NAME'].str.lower()

# Initialize flag/keyword column
df['Flag'] = None

# Defining regex patterns for start and end matching
start_patterns = [re.compile(r'\b{}.*'.format(keyword)) for keyword in fc]
end_patterns = [re.compile(r'.*\b{}'.format(keyword)) for keyword in fc]

# Iterating over each row and setting the flag
for index, row in df.iterrows():
    flags = []
    sentence = row['long_common_name_lower']
    for i, keyword in enumerate(fc):
        if start_patterns[i].match(sentence) or end_patterns[i].match(sentence):
            flags.append(keyword)
    if flags:
        df.at[index, 'Flag'] = ', '.join(flags)

# Properties we suspected that may contain PII
l = ['ClockTime', 'Date', 'URI', 'DtTmRange', 'ExtendedID', 'Addr', 'Angle', '-', 'Arb', 'Bib', 'Class', 'Cmplx', 'Desc', 'Find', 'Hx', 
    'ID', 'Imp', 'Instrct', 'Loc', 'DateRange', 'Pn', 'Prid', 'Tele', 'Time', 'TmStp','TmStpRange', 'TQ2', 'EmailAddr','Xad', 'Txt', 'Scope.modifier.selection item'
    , 'Scope.modifier.time window', 'Scope.modifier.doc template']

# Applying the property condition
condition = df['PROPERTY'].isin(l)
df_final = df[condition]

print('Number of LOINCs after applying the potential properties that may contain PII: ', df_final.shape[0])
print('Number of LOINCs after applying filtering through the keywords: ', df[df['Flag'].notna()].shape[0])

# Droping the lowercase version of long_common_name if no longer needed
out = df_final[['LOINC_NUM', 'PROPERTY', 'Flag', 'CLASS']]
fin = out[out['Flag'].notna()]

print('Number of LOINCs after applying both keywords and properties: ', fin[fin['Flag'].notna()].shape[0])

Total number of LOINCs:  104054
Number of LOINCs after applying the potential properties that may contain PII:  39997
Number of LOINCs after applying filtering through the keywords:  4752
Number of LOINCs after applying both keywords and properties:  4041


# Counting number of occurrence for each keyword/flag

In [4]:
value_counts = fin['Flag'].value_counts()
print(value_counts)

Flag
identifier                             1847
date                                    737
age                                     472
number                                  260
name                                    218
telephone                               116
address                                  81
name, identifier                         51
certificate                              40
number, phone                            36
city                                     16
phone                                    14
town                                     13
number, identifier                       13
license                                  12
address, email                           12
address, identifier                      10
postal                                    9
age, date                                 9
zip                                       9
county                                    8
longitude                                 6
latitude                   

In [ ]:
'''
After filtering for 4041 LOINCs: For the first column in Table 2 of the paper:

-- identifier                             1847+0 to identifier
    

-- date                                    737+0 to date

-- age                                     472+10 to age
    -- age, identifier                           1 to age
    -- age, date                                 9 to age

-- number                                  260+18 to number
    -- number, identifier                       13 to number
    -- number, age                               5 to number


-- name                                    218+53 to name
    -- name, identifier                         51- to name
    -- name, age                                 1 to name
    -- nickname                                  1 to name


-- telephone                               116+54 to phone
    -- phone                                    14 to phone
    -- number, phone                            36- to phone
    -- phone, date                               1 to phone
    -- number, telephone                         3 to phone

    
-- address                                  81+12 to address
    -- address, identifier                      10 to address
    -- address, name                             2 to address



-- certificate                              40+2 to certificate
    -- date, certificate                         1 to certificate
    -- number, certificate                       1 to certificate

-- city                                     16+5 to city
    -- city, town                                4 to city
    -- address, city                             1 to city



-- town                                     13+0 to town


-- license                                  12+5to license
    -- identifier, license                       1 to license
    -- age, license                              1 to license
    -- license, certificate                      1 to lisence
    -- date, license                             2 to lisence
    
-- postal                                    9+0 to postal


-- zip                                       9+0 to zip


-- county                                    8+2 to county
    -- date, county                              1 to county
    -- identifier, county                        1 to county
    
-- longitude                                 6+1 to longitude
    -- longitude, identifier                     1 to longitude

    
-- latitude                                  6+1 to latitude
    -- latitude, identifier                      1 to latitude
    

-- email                                     2+13 to email
    -- address, email                           12 - to email
    -- address, number, phone, email             1 to email
    
-- social security                           2+6 to SS
    -- number, social security, date             1 to social security
    -- number, social security                   4 to social security
    -- number, social security, identifier       1 to social security

    
-- url                                       1+1 to url
    -- address, url                              1 to url

    
-- fax                                       0+3 to url
    -- number, fax                               3 to fax
    
'''

# 1000 codes after review of three physicians

In [5]:
# First round of review
fin_review = pd.read_csv('Final decision - LOINC(Sheet1).csv')

# Second round of review
with open('supress-list.txt', 'r') as file:
    file_contents = [line.strip() for line in file]


loinc_values = fin_review['LOINC_NUM'].tolist()

# All chosen LOINCs
combined_list = file_contents + loinc_values

# Preparing the final data frame with LOINC_NUM, Porperty, CLASS and Flag feature

In [6]:
filtered_df = df_final[df_final['LOINC_NUM'].isin(combined_list)]
final = filtered_df[['LOINC_NUM', 'COMPONENT', 'PROPERTY', 'CLASS', 'Flag']]

# Counting number of occurrence for each CLASS and flag

In [7]:
value_counts = df['CLASS'].value_counts()
print(len(value_counts), value_counts)

427 CLASS
MICRO                   14527
CHEM                    10781
DRUG/TOX                 8804
RAD                      7292
ALLERGY                  5509
                        ...  
SURVEY.PAS                  1
PANEL.SURVEY.RSS            1
PANEL.SURVEY.ADVAULT        1
PANEL.SURVEY.NICHQ          1
PANEL.SURVEY.PAS            1
Name: count, Length: 427, dtype: int64


In [8]:
value_counts = df['Flag'].value_counts()
print(value_counts)

Flag
identifier                             1852
date                                    746
age                                     697
number                                  666
name                                    226
telephone                               119
address                                  85
name, identifier                         51
certificate                              44
license                                  37
number, phone                            36
city                                     17
zip                                      16
phone                                    14
town                                     13
number, identifier                       13
address, email                           12
county                                   10
address, identifier                      10
age, date                                 9
postal                                    9
number, age                               8
city, town                 

In [9]:
value_counts = final['PROPERTY'].value_counts()
print(len(value_counts), value_counts)

17 PROPERTY
Date          381
ID            171
TmStp         160
Pn             81
DateRange      35
Find           34
Tele           29
Addr           28
Loc            22
Cmplx          21
-              15
Angle           8
EmailAddr       8
ExtendedID      3
Prid            2
Xad             1
Txt             1
Name: count, dtype: int64


In [ ]:
'''

After filtering: For the second column in the paper
D-- identifier                             83+8 to identifier
D    -- name, identifier                         8- to identifier

D-- date                                    593 to date


D-- age                                     1+4 to age
D    -- age, date                                 1 to age
D    -- number, age                               3 to age


D-- number                                  64+2 to number
D    -- number, identifier                       2 to number


D-- name                                    107+1 to name
D    -- nickname                                  1 to name


D-- telephone                               4+27 to phone
D    -- phone                                    2 to phone
D    -- number, phone                            23- to phone
D    -- phone, date                               1 to phone
D    -- number, telephone                         1 to phone

    
D-- address                                  27+2 to address
D    -- address, name                             2 to address



D-- certificate                              18+2 to certificate
D    -- date, certificate                         1 to certificate
D    -- number, certificate                       1 to number


D-- city                                     8+1 to city
D    -- address, city                             1 to address


-- town                                     0+0 to town


D-- license                                  4+4 to license
D    -- identifier, license                       1 to license
D    -- license, certificate                      1 to lisence
D    -- date, license                             2 to date

    
D-- postal                                    7+0 to postal


D-- zip                                       4+0 to zip


D-- county                                    3+1 to county
D    -- date, county                              1 to county
    
D-- longitude                                 4+1 to longitude
D    -- longitude, identifier                     1 to longitude

    
D-- latitude                                  4+1 to latitude
D    -- latitude, identifier                      1 to latitude
    

D-- email                                     1+7 to email
D    -- address, email                           6 - to email
D    -- address, number, phone, email             1 to email
    
D-- social security                           0+4 to SS
D    -- number, social security, date             1 to social security
D    -- number, social security                   2 to social security
D    -- number, social security, identifier       1 to social security

    
D-- url                                       0+1 to url
D    -- address, url                              1 to url

    
D-- fax                                       0+2 to url
D    -- number, fax                               2 to fax
'''

In [10]:
final.to_csv('PII-LOINCs-list.csv', index=False)